In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


# error rate
def error_rate(xtrain, ytrain, x, opts):
    # parameters
    k     = opts['k']
    fold  = opts['fold']
    xt    = fold['xt']
    yt    = fold['yt']
    xv    = fold['xv']
    yv    = fold['yv']

    # Number of instances
    num_train = np.size(xt, 0)
    num_valid = np.size(xv, 0)
    # Define selected features
    xtrain  = xt[:, x == 1]
    ytrain  = yt.reshape(num_train)  # Solve bug
    xvalid  = xv[:, x == 1]
    yvalid  = yv.reshape(num_valid)  # Solve bug
    # Training
    mdl     = KNeighborsClassifier(n_neighbors = k)
    mdl.fit(xtrain, ytrain)
    # Prediction
    ypred   = mdl.predict(xvalid)
    acc     = np.sum(yvalid == ypred) / num_valid
    error   = 1 - acc

    return error


# Error rate & Feature size
def Fun(xtrain, ytrain, x, opts):
    # Parameters
    alpha    = 0.99
    beta     = 1 - alpha
    # Original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost  = 1
    else:
        # Get error rate
        error = error_rate(xtrain, ytrain, x, opts)
        # Objective function
        cost  = alpha * error + beta * (num_feat / max_feat)

    return cost

In [3]:
import numpy as np
from numpy.random import rand
#from FS.functionHO import Fun


def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()

    return X


def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            if X[i,d] > thres:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0

    return Xbin


def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub

    return x


#--- Opposition based learning (7)
def opposition_based_learning(X, lb, ub, thres, N, dim):
    Xo = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            Xo[i,d] = lb[0,d] + ub[0,d] - X[i,d]

    return Xo


def jfs(xtrain, ytrain, opts):
    # Parameters
    ub             = 1
    lb             = 0
    thres          = 0.5
    max_local_iter = 10     # maximum iteration for local search

    N              = opts['N']
    max_iter       = opts['T']
    if 'maxLt' in opts:
        max_local_iter = opts['maxLt']

    # Dimension
    dim = np.size(xtrain, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')

    # Initialize position
    X    = init_position(lb, ub, N, dim)

    # Pre
    fit  = np.zeros([N, 1], dtype='float')
    Xf   = np.zeros([1, dim], dtype='float')
    fitF = float('inf')

    # Binary conversion
    Xbin = binary_conversion(X, thres, N, dim)

    # Fitness
    for i in range(N):
        fit[i,0] = Fun(xtrain, ytrain, Xbin[i,:], opts)
        if fit[i,0] < fitF:
            Xf[0,:] = X[i,:]
            fitF    = fit[i,0]

    #--- Opposition based learning
    Xo    = opposition_based_learning(X, lb, ub, thres, N, dim)
    #--- Binary conversion
    Xobin = binary_conversion(Xo, thres, N, dim)

    #--- Fitness
    fitO  = np.zeros([N, 1], dtype='float')
    for i in range(N):
        fitO[i,0] = Fun(xtrain, ytrain, Xobin[i,:], opts)
        if fitO[i,0] < fitF:
            Xf[0,:] = Xo[i,:]
            fitF    = fitO[i,0]

    #--- Merge opposite & current population, and select best N
    XX  = np.concatenate((X, Xo), axis=0)
    FF  = np.concatenate((fit, fitO), axis=0)
    #--- Sort in ascending order
    ind = np.argsort(FF, axis=0)
    for i in range(N):
        X[i,:]   = XX[ind[i,0],:]
        fit[i,0] = FF[ind[i,0]]

    curve = np.zeros([1, max_iter], dtype='float')
    t     = 0

    # Store result
    curve[0,t] = fitF.copy()
    print("Iteration:", t + 1)
    print("Best (ISSA):", curve[0,t])
    t += 1

    while t < max_iter:
 	    # Compute coefficient, c1 (2)
        c1 = 2 * np.exp(-(4 * t / max_iter) ** 2)

        for i in range(N):
            # First leader update
            if i == 0:
                for d in range(dim):
                    # Coefficient c2 & c3 [0 ~ 1]
                    c2 = rand()
                    c3 = rand()
              	    # Leader update (1)
                    if c3 >= 0.5:
                        X[i,d] = Xf[0,d] + c1 * ((ub[0,d] - lb[0,d]) * c2 + lb[0,d])
                    else:
                        X[i,d] = Xf[0,d] - c1 * ((ub[0,d] - lb[0,d]) * c2 + lb[0,d])

                    # Boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])

            # Salp update
            elif i >= 1:
                for d in range(dim):
                    # Salp update by following front salp (3)
                    X[i,d] = (X[i,d] + X[i-1, d]) / 2
                    # Boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])

        # Binary conversion
        Xbin = binary_conversion(X, thres, N, dim)

        # Fitness
        for i in range(N):
            fit[i,0] = Fun(xtrain, ytrain, Xbin[i,:], opts)
            if fit[i,0] < fitF:
                Xf[0,:] = X[i,:]
                fitF    = fit[i,0]

        #--- Local search algorithm
        Lt        = 0
        temp      = np.zeros([1, dim], dtype='float')
        temp[0,:] = Xf[0,:]

        while Lt < max_local_iter:
            #--- Random three features
            RD = np.random.permutation(dim)
            for d in range(3):
                index = RD[d]
                #--- Flip the selected three features
                if temp[0,index] > thres:
                    temp[0,index] = temp[0,index] - thres
                else:
                    temp[0,index] = temp[0,index] + thres

            #--- Binary conversion
            temp_bin = binary_conversion(temp, thres, 1, dim)

            #--- Fitness
            Fnew = Fun(xtrain, ytrain, temp_bin[0,:], opts)
            if Fnew < fitF:
                fitF    = Fnew
                Xf[0,:] = temp[0,:]

            Lt += 1


        # Store result
        curve[0,t] = fitF.copy()
        print("Iteration:", t + 1)
        print("Best (ISSA):", curve[0,t])
        t += 1


    # Best feature subset
    Gbin       = binary_conversion(Xf, thres, 1, dim)
    Gbin       = Gbin.reshape(dim)
    pos        = np.asarray(range(0, dim))
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    issa_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}

    return issa_data

In [4]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
data0  = pd.read_csv('H:\Hybrid Metaheuristic Feature Selection\Dataset\IoT ID 20_StatisticalFS.csv',low_memory=False)
#read dataset. Use your own dataset path here
data0

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Idle_Std,Label,Cat
0,12446,25883,8006,203,76,2,3496,75,1,0,...,0,0,0,0,0,0,0,0,0,2
1,22760,34617,1186,200,10,1,3664,5297,1,1,...,0,0,0,2341,0,0,0,22632,0,0
2,12691,25886,9854,200,70,1,2082,141,0,2,...,0,0,0,748,0,0,0,253,0,4
3,12704,25886,9916,200,70,1,791,151,0,1,...,0,0,0,748,0,0,0,0,0,2
4,611,25881,8443,317,13,2,1040,153,2,0,...,0,0,0,0,0,0,0,253,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,62081,25889,10261,233,59,2,3443,277,1,0,...,0,0,0,0,0,0,0,0,0,2
625779,18760,30623,3574,200,10,1,3637,1648,0,1,...,0,0,0,2341,0,0,0,0,0,0
625780,12695,25886,9866,200,70,1,2285,77,1,0,...,0,0,0,3276,0,0,0,0,0,4
625781,12682,25883,8002,203,447,1,2872,240,2,0,...,0,0,0,748,0,0,0,6420,1,3


In [6]:
data0['Label'].value_counts()

Label
0    585710
1     40073
Name: count, dtype: int64

In [7]:
data1 = data0[(data0['Label']==1)]
data1 = data1.sample(n=None, frac=1,replace=False,weights=None,random_state=42,axis=0)

data2 = data0[(data0['Label']==0)]
data2 = data2.sample(n=None, frac=0.7,replace=False,weights=None,random_state=42,axis=0)

data0 = pd.concat([data1,data2])
data0

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Idle_Std,Label,Cat
336733,12682,25883,8002,203,447,1,2816,196,0,2,...,0,0,0,3204,0,0,0,10963,1,3
385404,47157,25889,9404,10,9,1,2910,318,0,1,...,0,0,0,401,0,0,0,0,1,3
513047,12683,25886,9277,200,70,1,2696,433,1,0,...,0,0,0,3365,0,0,0,0,1,3
534342,12682,25883,8002,203,447,1,2762,86,0,1,...,0,0,0,3204,0,0,0,0,1,3
44163,12682,25883,8002,203,447,1,2845,263,1,0,...,0,0,0,748,0,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124186,12447,25883,10368,203,76,2,3327,267,1,0,...,0,0,0,0,0,0,0,0,0,2
64005,46571,57931,4,203,817,1,3299,158,2,0,...,0,0,0,93,0,0,0,0,0,2
457240,62694,25889,11192,233,74,2,3193,2,1,0,...,0,0,0,0,0,0,0,0,0,2
324610,13761,25883,10911,233,67,2,3276,10,1,0,...,0,0,0,0,0,0,0,0,0,2


In [8]:
data0['Label'].value_counts()

Label
0    409997
1     40073
Name: count, dtype: int64

In [9]:
# z = data.columns
# for i in z:
#   print('Val: \n',data[i].value_counts(),'\n\n')

In [10]:
printdata = data0 #the dataframe used for displaying feature names since everthing will be in values now
printdata.to_csv('H:/Hybrid Metaheuristic Feature Selection/Dataset/IoT ID 20_GetColumnNames.csv',index=False)

In [11]:
data  = data0.values
print(data)

[[12682 25883  8002 ... 10963     1     3]
 [47157 25889  9404 ...     0     1     3]
 [12683 25886  9277 ...     0     1     3]
 ...
 [62694 25889 11192 ...     0     0     2]
 [13761 25883 10911 ...     0     0     2]
 [62080 25889 10260 ...     0     0     2]]


In [12]:
#seperate the features and labels
feat  = np.asarray(data[:, 0:-1])   # feature vector
feat

array([[12682, 25883,  8002, ...,     0, 10963,     1],
       [47157, 25889,  9404, ...,     0,     0,     1],
       [12683, 25886,  9277, ...,     0,     0,     1],
       ...,
       [62694, 25889, 11192, ...,     0,     0,     0],
       [13761, 25883, 10911, ...,     0,     0,     0],
       [62080, 25889, 10260, ...,     0,     0,     0]], dtype=int64)

In [13]:
label = np.asarray(data[:, -2])     # label vector
label

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [14]:
# split data into train & validation (70 -- 30)
xtrain, xtest, ytrain, ytest = train_test_split(feat, label, test_size=0.3, stratify=label)
fold = {'xt':xtrain, 'yt':ytrain, 'xv':xtest, 'yv':ytest}

# parameter
k     = 5     # k-value in KNN
N     = 34    # number of salps
T     = 2   # maximum number of iterations
maxLt = 4    # maximum iteration for local search algorithm
opts  = {'k':k, 'fold':fold, 'N':N, 'T':T, 'maxLt':maxLt}

# perform feature selection
fmdl  = jfs(feat, label, opts)
sf    = fmdl['sf']

# model with selected features
num_train = np.size(xtrain, 0)
num_valid = np.size(xtest, 0)
x_train   = xtrain[:, sf]
y_train   = ytrain.reshape(num_train)  # Solve bug
x_valid   = xtest[:, sf]
y_valid   = ytest.reshape(num_valid)  # Solve bug

mdl = KNeighborsClassifier(n_neighbors = k)
mdl.fit(x_train, y_train)

# accuracy
y_pred    = mdl.predict(x_valid)
Acc       = np.sum(y_valid == y_pred)  / num_valid
print("Accuracy:", 100 * Acc)

# number of selected features
num_feat = fmdl['nf']
print("Feature Size:", num_feat)
print("Feature selected:", sf)
# plot convergence
curve   = fmdl['c']
curve   = curve.reshape(np.size(curve,1))
x       = np.arange(0, opts['T'], 1.0) + 1.0

fig, ax = plt.subplots()
ax.plot(x, curve, 'o-')
ax.set_xlabel('Number of Iterations')
ax.set_ylabel('Fitness')
ax.set_title('ISSA')
ax.grid()
plt.show()

KeyboardInterrupt: 

In [ ]:
for i in sf:
  print(printdata.columns[i])

: 

In [ ]:

import seaborn as sns

x_train_df = pd.DataFrame(x_train)
x_valid_df = pd.DataFrame(x_valid)

# Calculate correlation matrix
corrmat = x_train_df.corr(method='pearson')
plt.figure(figsize=(10,10))
sns.heatmap(corrmat,annot=True,linewidth=.5,fmt=".1f",cmap=plt.cm.Reds)
plt.show()

# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(pd.DataFrame(x_train), 0.7)
len(set(corr_features))

corr_features

x_train_corr=pd.DataFrame(x_train).drop(corr_features,axis=1)
x_test_corr=pd.DataFrame(x_valid).drop(corr_features,axis=1)

x_train_corr.shape,x_test_corr.shape,y_train.shape,y_valid.shape

: 

In [ ]:
selected_columns_index = sf

: 

In [ ]:
selected_columns = printdata.iloc[:, selected_columns_index]
selected_columns

: 

In [ ]:
label_index = printdata.columns.get_loc('Label')

: 

In [ ]:
label_new = printdata.iloc[:, label_index]
label_new

: 

In [ ]:
selected_columns[len(selected_columns.columns)+1] = label_new
selected_columns

: 

In [ ]:
new_df = pd.DataFrame(selected_columns)
new_df.to_csv('E:/Hybrid Metaheuristic Feature Selection/Dataset/IoT ID 20_MHFS.csv',index=False)

: 